In [ ]:
import pandas as pd
import pickle
import os
import numpy as np
# data = pickle.load(open('../data/leftppgecg.p','rb'))
directory = '../data_users/ecg_ppg/'
dfs = []
for f in os.listdir(directory)[:50]:
    if f[-1]!='p':
        continue
    a = pickle.load(open(directory+f,'rb'))
    print(a.shape,end=',')
    dfs.append(a)
print()
data1 = pd.concat(dfs)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from joblib import Parallel,delayed
from hrvanalysis import get_time_domain_features

from copy import deepcopy
data_all = deepcopy(data1)

data_all['red_rr'] = data_all['ppg_rr'].apply(lambda x:x[0])
data_all['ir_rr'] = data_all['ppg_rr'].apply(lambda x:x[1])
data_all['green_rr'] = data_all['ppg_rr'].apply(lambda x:x[2])
data_all['red_qual'] = data_all['likelihood'].apply(lambda x:x[0])
data_all['ir_qual'] = data_all['likelihood'].apply(lambda x:x[1])
data_all['green_qual'] = data_all['likelihood'].apply(lambda x:x[2])
data_all['index'] = data_all['likelihood'].apply(lambda x:np.argmax(np.array(x)))
values = data_all[['ppg_rr','index']].values
values = [a[b] for a,b in values]
data_all['ppg_rr_best'] = values
data_all['likelihood_best'] = data_all['likelihood'].apply(lambda x:max(x))

In [ ]:
import pickle
pickle.dump(data_all,open('../data_users/merged_data.p','wb'),protocol=4)

In [1]:
import pandas as pd
import pickle
import os
import numpy as np
data_all = pickle.load(open('../data_users/merged_data.p','rb'))

In [2]:
import warnings
warnings.filterwarnings('ignore')
from joblib import Parallel,delayed
from hrvanalysis import get_time_domain_features

from copy import deepcopy
def get_data1(a):
    features = []
    ecg_rr = a[:,-1]
    if len(ecg_rr[np.isnan(ecg_rr)])>0:
        return [],[],[],[],[],[]
    m = np.mean(ecg_rr[ecg_rr>0])
    s = np.std(ecg_rr[ecg_rr>0])
    ecg_rr[np.isnan(ecg_rr)] = 0
    if len(ecg_rr)<60:
        return [],[],[],[],[],[]
    y = []
    X = []
    ecg = []
    means = []
    stds = []
    quals = []
    for i in [-2]:
        ppg_rr = a[:,i]
        ppg_qual = a[:,i-4]
#         index = ppg_rr>0
#         ppg_qual = ppg_qual[index]
#         ppg_rr = ppg_rr[index]
        index = np.isnan(ppg_rr)
        index1 = ~np.isnan(ppg_rr)
        if len(ppg_rr[index1])<60:
            continue
        ppg_qual[index] = -1
        ppg_rr[index] = np.nanmean(ppg_rr)
        y.append(ecg_rr.reshape(1,60,1))
        tmp = a[:,np.array([-2,-3,-4,-5,-6,-7,-8,-9,1])].reshape(1,60,9)
        tmp[np.isnan(tmp)] = 0
        tmp[tmp==0] = 0
        X.append(tmp)
        means.append(m)
        stds.append(s)
        ecg.append(ppg_rr.reshape(1,60,1))
        quals.append(ppg_qual.reshape(1,60,1))
#         for j in np.linspace(0,.9,20):
#             index = ppg_qual>j
#             ppg_qual = ppg_qual[index]
#             ppg_rr = ppg_rr[index]
#             if len(ppg_rr)<10:
#                 continue
#             f = list(get_time_domain_features(ppg_rr).values())
#             f1 = list(get_time_domain_features(ecg_rr).values())
#             q = [np.percentile(ppg_qual,20),np.median(ppg_qual),len(ppg_rr)/60]
#             features.append(np.array(f1+f+q))
    return X,y,ecg,means,stds,quals

unique_users = data_all['user'].unique()

def get_data(name,df):
    df = df[['time','activity','red_qual','ir_qual','green_qual','likelihood_best',
             'red_rr','ir_rr','green_rr','ppg_rr_best','ecg_rr']].values
    return df.reshape(-1,60,11)
all_X = []
for user in unique_users:
    data_user = data_all[data_all.user.isin([user])]
    data_user.set_index('timestamp',inplace=True)
    convert_dict = {'ecg_rr': float}
    data_user = data_user.astype(convert_dict) 
    data_resampled = data_user.resample('1S').mean()
    if 'ecg_rr' not in np.array(data_resampled.columns.values):
        continue
    df_col = Parallel(n_jobs=20,verbose=1)(delayed(get_data)(group_name, df_group) for group_name, df_group
                                           in data_resampled.groupby(pd.Grouper(freq='60S')) if df_group.shape[0]==60)
#     df_col = [get_data(group_name, df_group) for group_name, df_group
#                                            in data_resampled.groupby(pd.Grouper(freq='60S')) if df_group.shape[0]==60]
    df_user = np.concatenate(df_col)
    df_col = Parallel(n_jobs=20,verbose=1)(delayed(get_data1)(a) for a in df_user if len(a[~np.isnan(a[:,-1]),-1])>20)    
    all_X.extend(df_col)
    print(user)

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    4.1s
[Parallel(n_jobs=20)]: Done 241 out of 241 | elapsed:    4.5s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 182 out of 182 | elapsed:    0.3s finished


44514a20-82cf-45ee-8fae-828f2ec4f035


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 600 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 2600 tasks      | elapsed:    3.0s
[Parallel(n_jobs=20)]: Done 5400 tasks      | elapsed:    5.4s
[Parallel(n_jobs=20)]: Done 9000 tasks      | elapsed:    8.8s
[Parallel(n_jobs=20)]: Done 13400 tasks      | elapsed:   13.0s
[Parallel(n_jobs=20)]: Done 18600 tasks      | elapsed:   17.9s
[Parallel(n_jobs=20)]: Done 19305 out of 19305 | elapsed:   18.7s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   8 out of  23 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=20)]: Done  23 out of  23 | elapsed:    0.1s finished


3133920d-164a-48e6-8d9e-79e919c45d43


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 600 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 2600 tasks      | elapsed:    2.8s
[Parallel(n_jobs=20)]: Done 5400 tasks      | elapsed:    5.4s
[Parallel(n_jobs=20)]: Done 9000 tasks      | elapsed:    8.8s
[Parallel(n_jobs=20)]: Done 11645 out of 11645 | elapsed:   11.1s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 600 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 1104 out of 1104 | elapsed:    1.0s finished


...

[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 600 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 2600 tasks      | elapsed:    2.8s
[Parallel(n_jobs=20)]: Done 5400 tasks      | elapsed:    5.5s
[Parallel(n_jobs=20)]: Done 9000 tasks      | elapsed:    8.9s
[Parallel(n_jobs=20)]: Done 13400 tasks      | elapsed:   13.2s
[Parallel(n_jobs=20)]: Done 18600 tasks      | elapsed:   18.1s
[Parallel(n_jobs=20)]: Done 19480 out of 19480 | elapsed:   18.7s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 600 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 6690 tasks      | elapsed:    1.8s
[Parallel(n_jobs=20)]: Done 7032 out of 7071 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=20)]: Done 7071 out of 7071 | elapsed:    2

fdddb3bd-bb88-458f-bcc8-e50bb3f87742


[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 600 tasks      | elapsed:    1.1s
[Parallel(n_jobs=20)]: Done 2600 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 5400 tasks      | elapsed:    5.4s
[Parallel(n_jobs=20)]: Done 9000 tasks      | elapsed:    8.8s
[Parallel(n_jobs=20)]: Done 13400 tasks      | elapsed:   13.0s
[Parallel(n_jobs=20)]: Done 18600 tasks      | elapsed:   18.0s
[Parallel(n_jobs=20)]: Done 23117 out of 23117 | elapsed:   22.3s finished
[Parallel(n_jobs=20)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 600 tasks      | elapsed:    0.5s
[Parallel(n_jobs=20)]: Done 3310 tasks      | elapsed:    1.5s


02543bbf-84c2-4076-8547-c8a5f451ea02


[Parallel(n_jobs=20)]: Done 3519 out of 3519 | elapsed:    1.9s finished


In [3]:
X,y,ecg,means,stds,quals = [],[],[],[],[],[]
for a in all_X:
    if len(a[0])==0:
        continue
    X.extend(a[0])
    y.extend(a[1])
    ecg.extend(a[2])
    means.extend(a[3])
    stds.extend(a[4])
    quals.extend(a[5])

In [4]:
import pickle
pickle.dump([X,y,ecg,means,stds,quals],open('../data_users/processed_data.p','wb'),protocol=4)

In [5]:
import pandas as pd
import pickle
import os
import numpy as np
X,y,ecg,means,stds,quals = pickle.load(open('../data_users/processed_data.p','rb'))

In [6]:
X,y,ecg,means,stds,quals = np.concatenate(X),np.concatenate(y).reshape(-1,60),np.concatenate(ecg).reshape(-1,60),\
np.array(means).reshape(-1,1),np.array(stds).reshape(-1,1),np.concatenate(quals)

In [7]:
from keras.layers import Input, LSTM, RepeatVector,Bidirectional,Multiply,multiply,Permute
from keras.layers import TimeDistributed,Dense,Flatten,Reshape,Lambda,Activation,GRU
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np
import keras.backend as K
from keras import metrics,losses
import tensorflow as tf
# import tensorflow_probability as tfp
X_train, X_test, y_train, y_test,ecg_train, \
ecg_test,means_train,means_test,stds_train,stds_test, \
quals_train,quals_test= train_test_split(
    X[:,:,np.array([0,1,2,3,4])], y,ecg,means,stds,quals, test_size=0.33, random_state=42)
X_train, X_val, y_train, y_val,means_train,means_val,stds_train,stds_val = train_test_split(
    X_train, y_train,means_train,stds_train, test_size=0.2, random_state=42)
print(X_train.shape,y_train.shape,y_val.shape,means_val.shape,stds_val.shape)

Using TensorFlow backend.


(6786, 60, 5) (6786, 60) (1697, 60) (1697, 1) (1697, 1)


In [8]:
def output_of_lambda(input_shape):
    return (input_shape[0], 1)

def mean(x):
    return K.mean(x, axis=1, keepdims=True)

def output_of_lambda1(input_shape):
    return (input_shape[0], 1)

def mean1(x):
    return K.std(x, axis=1, keepdims=True)

timesteps = 60
input_dim = 5
latent_dim = 20
output_dim = 1
n = 1
inputs = Input(shape=(timesteps, input_dim))
# inputs2 = Reshape((1,1))(inputs1)
encoded = Bidirectional(GRU(60,return_sequences=True,activation='relu',go_backwards=True))(inputs)
# encoded = LSTM(output_dim,return_sequences=True,activation='sigmoid')(encoded)
att = Dense(1,activation='relu')(encoded)

att = Flatten()(att)
att = Activation(activation="softmax")(att)
att = RepeatVector(120)(att)
att = Permute((2,1))(att)
mer = multiply([att, encoded])

encoded = TimeDistributed(Dense(1,activation='relu'))(mer)
encoded = Flatten()(encoded)
# encoded = Dense(10,activation='relu',name='sequence1')(encoded)
encoded = Dense(60,activation='relu',name='sequence')(encoded)
# encoded = Reshape((60),name='sequence')(encoded)
# encoded_std = K.std(encoded,axis=1)
decoded = Lambda(mean, output_shape=output_of_lambda,name='mean')(encoded)
decoded1 = Lambda(mean1, output_shape=output_of_lambda1,name='std')(encoded)
# decoded = LSTM(output_dim*60, return_sequences=True)(decoded)
# decoded = LSTM(output_dim*3, return_sequences=True)(decoded)
# decoded = LSTM(output_dim, return_sequences=True)(decoded)

sequence_autoencoder = Model(inputs=[inputs], outputs=[encoded,decoded,decoded1])
# encoder = Model(inputs, encoded)
losses = {
    "std":"mae",
    "mean": "mae",
    "sequence": "mae"
}
lossWeights = {"mean": 0, "sequence": 1,"std":0}
# initialize the optimizer and compile the model

sequence_autoencoder.compile(optimizer='adam',loss=losses, loss_weights=lossWeights)

sequence_autoencoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 60, 5)        0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 60, 120)      23760       input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 60, 1)        121         bidirectional_1[0][0]            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 60)           0           dense_1[0][0]                    
____________________________________________________________________________________________

In [ ]:
filepath = '../models/base_LSTM.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=30)
callbacks_list = [es,checkpoint]
history = sequence_autoencoder.fit(X_train, [y_train,means_train,stds_train],
                epochs=300,
                batch_size=100,
                shuffle=True,
                validation_data=(X_val,[y_val,means_val,stds_val]),callbacks=callbacks_list)

Train on 6786 samples, validate on 1697 samples
Epoch 1/300
6786/6786 [==============================] - 7s 1ms/step - loss: 634.9004 - sequence_loss: 634.7698 - mean_loss: 634.7698 - std_loss: 36.2917 - val_loss: 568.8140 - val_sequence_loss: 568.8242 - val_mean_loss: 568.8242 - val_std_loss: 94.8834

Epoch 00001: val_loss improved from inf to 568.81398, saving model to ../models/base_LSTM.hdf5
Epoch 2/300
6786/6786 [==============================] - 6s 916us/step - loss: 459.2380 - sequence_loss: 459.0777 - mean_loss: 447.2945 - std_loss: 186.6703 - val_loss: 386.2578 - val_sequence_loss: 386.2764 - val_mean_loss: 354.4275 - val_std_loss: 242.8112

Epoch 00002: val_loss improved from 568.81398 to 386.25778, saving model to ../models/base_LSTM.hdf5
Epoch 3/300
6786/6786 [==============================] - 6s 884us/step - loss: 336.2216 - sequence_loss: 336.1483 - mean_loss: 291.6076 - std_loss: 248.8277 - val_loss: 300.5981 - val_sequence_loss: 300.6118 - val_mean_loss: 244.4496 - val_

6786/6786 [==============================] - 6s 912us/step - loss: 86.0214 - sequence_loss: 86.0147 - mean_loss: 81.0001 - std_loss: 26.4784 - val_loss: 89.3087 - val_sequence_loss: 89.3140 - val_mean_loss: 84.5219 - val_std_loss: 26.3888

Epoch 00025: val_loss did not improve from 88.25760
Epoch 26/300
6786/6786 [==============================] - 6s 910us/step - loss: 84.3739 - sequence_loss: 84.3807 - mean_loss: 79.3033 - std_loss: 26.5169 - val_loss: 86.2915 - val_sequence_loss: 86.2972 - val_mean_loss: 81.3618 - val_std_loss: 26.6205

Epoch 00026: val_loss improved from 88.25760 to 86.29145, saving model to ../models/base_LSTM.hdf5
Epoch 27/300
6786/6786 [==============================] - 6s 914us/step - loss: 83.5520 - sequence_loss: 83.5300 - mean_loss: 78.3518 - std_loss: 26.5878 - val_loss: 85.4789 - val_sequence_loss: 85.4830 - val_mean_loss: 80.4638 - val_std_loss: 26.6815

Epoch 00027: val_loss improved from 86.29145 to 85.47894, saving model to ../models/base_LSTM.hdf5
Epoc

6786/6786 [==============================] - 6s 914us/step - loss: 74.2833 - sequence_loss: 74.2655 - mean_loss: 68.1311 - std_loss: 27.3171 - val_loss: 78.9675 - val_sequence_loss: 78.9563 - val_mean_loss: 73.8121 - val_std_loss: 27.2567

Epoch 00050: val_loss did not improve from 75.12620
Epoch 51/300
6786/6786 [==============================] - 6s 913us/step - loss: 72.1500 - sequence_loss: 72.1501 - mean_loss: 65.9412 - std_loss: 27.4833 - val_loss: 76.2006 - val_sequence_loss: 76.1891 - val_mean_loss: 70.5374 - val_std_loss: 27.5677

Epoch 00051: val_loss did not improve from 75.12620
Epoch 52/300
6786/6786 [==============================] - 6s 918us/step - loss: 73.1105 - sequence_loss: 73.1039 - mean_loss: 66.9552 - std_loss: 27.4045 - val_loss: 74.7801 - val_sequence_loss: 74.7681 - val_mean_loss: 68.9098 - val_std_loss: 27.4406

Epoch 00052: val_loss improved from 75.12620 to 74.78012, saving model to ../models/base_LSTM.hdf5
Epoch 53/300
6786/6786 [===========================

6786/6786 [==============================] - 6s 916us/step - loss: 71.0191 - sequence_loss: 70.9974 - mean_loss: 64.6621 - std_loss: 28.0280 - val_loss: 72.5670 - val_sequence_loss: 72.5578 - val_mean_loss: 66.2405 - val_std_loss: 27.9827

Epoch 00076: val_loss did not improve from 71.92927
Epoch 77/300
6786/6786 [==============================] - 6s 920us/step - loss: 70.2031 - sequence_loss: 70.2077 - mean_loss: 63.7862 - std_loss: 27.7644 - val_loss: 75.9619 - val_sequence_loss: 75.9546 - val_mean_loss: 70.3504 - val_std_loss: 27.6573

Epoch 00077: val_loss did not improve from 71.92927
Epoch 78/300
6786/6786 [==============================] - 6s 917us/step - loss: 70.3751 - sequence_loss: 70.3650 - mean_loss: 63.8670 - std_loss: 27.9604 - val_loss: 72.5399 - val_sequence_loss: 72.5310 - val_mean_loss: 66.1479 - val_std_loss: 28.1846

Epoch 00078: val_loss did not improve from 71.92927
Epoch 79/300
6786/6786 [==============================] - 6s 914us/step - loss: 70.8933 - sequence

6786/6786 [==============================] - 6s 911us/step - loss: 68.7657 - sequence_loss: 68.7881 - mean_loss: 61.9857 - std_loss: 28.0903 - val_loss: 71.8053 - val_sequence_loss: 71.7960 - val_mean_loss: 65.2196 - val_std_loss: 28.1542

Epoch 00102: val_loss did not improve from 71.06227
Epoch 103/300
6700/6786 [============================>.] - ETA: 0s - loss: 68.3142 - sequence_loss: 68.3142 - mean_loss: 61.4826 - std_loss: 28.0932

In [ ]:
from keras.models import load_model
sequence_autoencoder = load_model(filepath)

In [ ]:
y_pred1 = sequence_autoencoder.predict(X_test)
y_pred = y_pred1[0]
mean_pred = y_pred1[1]
stds_pred = y_pred1[2]

In [ ]:
y_pred1[0].shape,y_pred1[1].shape,y_pred1[2].shape

In [ ]:
%matplotlib  inline
import matplotlib.pyplot as plt
# plt.plot(y[-10])
for i,a in enumerate(y_pred[::-1][200:240]):
#     if np.sum(a)>0:
    plt.figure()
    plt.plot(a,'g')
#         plt.plot(X[i,:,:],'r')
    plt.plot(y_test[i],'b')
    plt.plot(ecg_test[i],'r')
    plt.show()

In [ ]:
X = np.zeros((len(quals_test.reshape(-1)),4))
X[:,0] = quals_test.reshape(-1)
X[:,1] = y_pred.reshape(-1)
X[:,2] = ecg_test.reshape(-1)
X[:,3] = y_test.reshape(-1)

In [ ]:
X = X[X[:,0]>-1]
X = X[X[:,2]>0]
X = X[X[:,3]>0]


In [ ]:
plt.hist(X[:,0])

In [ ]:
l_range = np.arange(0,1,.05)
x = []
y = []
y1 = []
for l in l_range:
    index = np.where((X[:,0]>=l)&(X[:,0]<l+.05))[0]
    temp = X[index]
    print(temp.shape,l)
    x.append(str(np.round(l*100)/100)+'-'+str(np.round((l+.05)*100)/100))
    y.append(list(np.abs(temp[:,2]-temp[:,3])))
    a = np.array(np.abs(temp[:,1]-temp[:,2]))
    y1.append(a[~np.isnan(a)])
print(len(y),len(y1))
#     print(np.mean(np.abs(temp[:,0]-temp[:,2])),np.std(np.abs(temp[:,0]-temp[:,2])),len(index))
plt.figure(figsize=(10,8))
plt.rcParams.update({'font.size':20})
c = plt.boxplot(y,showfliers=False,positions=np.array(range(0,3*len(y),3)),notch=True)
for box in c['boxes']:
    box.set(color='red', linewidth=1)
b = plt.boxplot(y1,showfliers=False,positions=np.array(range(0,3*len(y),3))+1.5,notch=True)
for box in b['boxes']:
    box.set(color='blue', linewidth=1)
#     box.set(facecolor = 'red' )
plt.xticks(np.array(range(0,3*len(y),3)),x,rotation=60)
plt.ylabel('Absolute Difference in Milliseconds')
plt.xlabel('Range of Signal Quality')
plt.tight_layout()
plt.show()   

In [ ]:
from hrvanalysis import get_time_domain_features
x = []
y = []
z = []
q = []
s = 'rmssd'
for i in range(ecg_test.shape[0]):
    qual_min = quals_test[i].reshape(-1)
    qual_min = qual_min[qual_min>-1]
    ecg_min = ecg_test[i].reshape(-1)
    ecg_min = ecg_min[ecg_min>0]
    ecg_min= ecg_min[~np.isnan(ecg_min)]
    y_pred_min = y_pred[i].reshape(-1)
    y_pred_min = y_pred_min[~np.isnan(y_pred_min)]
    y_test_min = y_test[i].reshape(-1)
    y_test_min = y_test_min[y_test_min>0]
    y_test_min = y_test_min[~np.isnan(y_test_min)]
    if len(ecg_min)<10 or len(y_pred_min)<5 or len(y_test_min)<5:
        continue
    x.append(get_time_domain_features(y_pred_min)[s])
    y.append(get_time_domain_features(ecg_min)[s])
    z.append(get_time_domain_features(y_test_min)[s])
    q.append(np.median(qual_min))
    if np.isinf(x[-1]) or np.isinf(y[-1]) or np.isinf(z[-1]) or np.isinf(q[-1]):
        x = x[:-1]
        y= y[:-1]
        z= z[:-1]
        q= q[:-1]
    elif np.isnan(x[-1]) or np.isnan(y[-1]) or np.isnan(z[-1]) or np.isnan(q[-1]):
        x = x[:-1]
        y= y[:-1]
        z= z[:-1]
        q= q[:-1]
#     print(np.std(ecg_min),np.std(y_test_min),np.std(y_pred_min),np.median(qual_min))
    
    

In [ ]:
from scipy.stats import pearsonr,spearmanr
q,x,y,z = np.array(q),np.array(x),np.array(y),np.array(z)
for i in np.linspace(0,.9,10):
    index = np.where((q>=i)&(q<=i+.1))[0]
    if len(index)<2:
        continue
    print(pearsonr(x[index],y[index]),pearsonr(y[index],z[index]),i,i+.1,len(index))

In [ ]:
# plt.hist(y.reshape(-1),50)
plt.hist(y_pred.reshape(-1),50)
plt.hist(ecg_test.reshape(-1),50)

In [ ]:
np.std(y[0].reshape(1,-1,1),axis=1)

In [ ]:
plt.plot(y_pred1[1].reshape(-1),means_test.reshape(-1),'*')

In [ ]:
from scipy.stats import pearsonr
pearsonr(y_pred1[1].reshape(-1),means_test.reshape(-1))

In [ ]:
import keras

In [ ]:
keras.__version__

In [ ]:
y_pred.reshape(-1).shape

In [ ]:
t = y_pred.reshape(-1)

In [ ]:
t[np.isfinite(t)].shape

In [ ]:
plt.hist(quals_test.reshape(-1))